# ニューラルネットワークにおける勾配

ここでいうニューラルネットワークにおける勾配というのは、重みパラメータに関する損失関数の勾配
勾配の要素は、それぞれの重みに関する偏微分から構成される
簡単なニューラルネットワークを例にして実際にやってみる

In [1]:
import sys, os

os.chdir('/users/isonoyuma/deep-learning-from-scratch/ch04')
sys.path.append(os.pardir)
import numpy as np

from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

In [10]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)#ガウス分布で初期化
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [11]:
net = simpleNet()
print(net.W)

[[-0.78425015  1.35311457  0.17493891]
 [-0.44785118 -0.55581795  0.26695799]]


In [12]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[-0.87361615  0.31163259  0.34522553]


In [13]:
np.argmax(p)

2

In [16]:
t = np.array([0, 0, 1])
net.loss(x, t)

0.8164866875612973

In [17]:
def f(W):
    return net.loss(x,t)

dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.07838251  0.25642842 -0.33481093]
 [ 0.11757377  0.38464263 -0.5022164 ]]


## 二層ニューラルネットワークの簡単な実装

In [21]:
import sys, os

os.chdir('/users/isonoyuma/deep-learning-from-scratch')
sys.path.append(os.pardir)
import numpy as np

from common.functions import *
from common.gradient import numerical_gradient

In [20]:
class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        
        self.params["b2"] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params["W1"], self.params["W2"]
        b1, b2 = self.params["b1"], self.params["b2"]
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    #x : 入力データ, t : 教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grads["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grads["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grads["b2"] = numerical_gradient(loss_W, self.params["b2"])
        
        return grads
    
    def gradient(self, x, t):
        pass#未実装